In [1]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from app.utils.visualization import plot_data
from app.utils.fetch_data import fred_fred, investing_api, alpha_vantage_api, alpha_vantage_api_financial_statements
from app.utils.fetch_data_fmp import FMP

In [3]:
pd.set_option('display.max_columns', None)

In [18]:
start_date = '2000-01-01'
today = date.today().strftime("%Y-%m-%d")

target_name = "sp500"
target_symbol = "%5EGSPC"
file_name = "master.csv" # save file name

In [19]:
fmp = FMP()

2022-10-01 18:27:56,515 [app.utils.fetch_data_fmp] [INFO] Financial Modeling Prep api ready.


## Gather data - target

In [20]:
target = fmp.get_historical_daily_price("index", target_symbol, start_date, today)
target.rename(columns={"symbol":target_name}, inplace=True)

2022-10-01 18:27:58,833 [app.utils.fetch_data_fmp] [INFO] Fetching daily price data of %5EGSPC


In [21]:
# sp500
target.tail()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,sp500
5719,2000-01-07,1403.449951,1441.469971,1400.729981,1441.469971,1441.469971,1.225200e+09,1.225200e+09,38.02002,2.709,1427.88997,"January 07, 00",0.02709,^GSPC
5720,2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1.092300e+09,1.092300e+09,1.33997,0.096,1402.48332,"January 06, 00",0.00096,^GSPC
5721,2000-01-05,1399.420044,1413.270019,1377.680054,1402.109985,1402.109985,1.085500e+09,1.085500e+09,2.68994,0.192,1397.68669,"January 05, 00",0.00192,^GSPC
5722,2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1.009000e+09,1.009000e+09,-55.79993,-3.834,1417.35669,"January 04, 00",-0.03834,^GSPC
5723,2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,9.318000e+08,9.318000e+08,-14.03003,-0.955,1457.19332,"January 03, 00",-0.00955,^GSPC


## Gather data - economic data (FMP)

In [22]:
econ_indices = pd.DataFrame(target['date'])

In [23]:
for i in ["GDP", "realGDP", "nominalPotentialGDP", "realGDPPerCapita",
        "federalFunds", "CPI", "inflationRate", "inflation", "retailSales", "consumerSentiment", "durableGoods",
        "unemploymentRate", "totalNonfarmPayroll", "initialClaims", "industrialProductionTotalIndex",
        "newPrivatelyOwnedHousingUnitsStartedTotalUnits", "totalVehicleSales", "retailMoneyFunds",
        "smoothedUSRecessionProbabilities", "3MonthOr90DayRatesAndYieldsCertificatesOfDeposit",
        "commercialBankInterestRateOnCreditCardPlansAllAccounts", "30YearFixedRateMortgageAverage",
        "15YearFixedRateMortgageAverage"]:
    df = fmp.get_economic_index(i, start_date, today)
    econ_indices = econ_indices.merge(df, on='date', how='left')

2022-10-01 18:28:03,164 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: GDP
2022-10-01 18:28:04,015 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: realGDP
2022-10-01 18:28:04,856 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: nominalPotentialGDP
2022-10-01 18:28:05,690 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: realGDPPerCapita
2022-10-01 18:28:06,597 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: federalFunds
2022-10-01 18:28:07,516 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: CPI
2022-10-01 18:28:08,346 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: inflationRate
2022-10-01 18:28:10,017 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: inflation
2022-10-01 18:28:10,851 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: retailSales
2022-10-01 18:28:11,721 [app.utils.fetch_data_fmp] [INFO] Fetching economic index data: consumerSentiment


In [24]:
econ_indices.tail(3)

,date,GDP,realGDP,nominalPotentialGDP,realGDPPerCapita,federalFunds,CPI,inflationRate,inflation,retailSales,consumerSentiment,durableGoods,unemploymentRate,totalNonfarmPayroll,initialClaims,industrialProductionTotalIndex,newPrivatelyOwnedHousingUnitsStartedTotalUnits,totalVehicleSales,retailMoneyFunds,smoothedUSRecessionProbabilities,3MonthOr90DayRatesAndYieldsCertificatesOfDeposit,commercialBankInterestRateOnCreditCardPlansAllAccounts,30YearFixedRateMortgageAverage,15YearFixedRateMortgageAverage
5721,2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5722,2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5723,2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge into master

In [25]:
master = target.merge(econ_indices, on='date', how='left')

In [26]:
master.head()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,sp500,GDP,realGDP,nominalPotentialGDP,realGDPPerCapita,federalFunds,CPI,inflationRate,inflation,retailSales,consumerSentiment,durableGoods,unemploymentRate,totalNonfarmPayroll,initialClaims,industrialProductionTotalIndex,newPrivatelyOwnedHousingUnitsStartedTotalUnits,totalVehicleSales,retailMoneyFunds,smoothedUSRecessionProbabilities,3MonthOr90DayRatesAndYieldsCertificatesOfDeposit,commercialBankInterestRateOnCreditCardPlansAllAccounts,30YearFixedRateMortgageAverage,15YearFixedRateMortgageAverage
0,2022-09-30,3633.4800,3671.4399,3584.1299,3585.6201,3585.6201,5.645543e+09,5.645543e+09,-47.8599,-1.317,3613.72997,"September 30, 22",-0.01317,^GSPC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-29,3687.0100,3687.0100,3610.3999,3640.4700,3640.4700,4.682252e+09,4.682252e+09,-46.5400,-1.262,3645.95997,"September 29, 22",-0.01262,^GSPC,NaN,NaN,NaN,NaN,NaN,NaN,2.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.7,5.96
2,2022-09-28,3651.9399,3736.7400,3640.6101,3719.0400,3719.0400,4.684890e+09,4.684890e+09,67.1001,1.837,3698.79670,"September 28, 22",0.01837,^GSPC,NaN,NaN,NaN,NaN,NaN,NaN,2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-27,3686.4399,3717.5300,3623.2900,3647.2900,3647.2900,4.578065e+09,4.578065e+09,-39.1499,-1.062,3662.70333,"September 27, 22",-0.01062,^GSPC,NaN,NaN,NaN,NaN,NaN,NaN,2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-26,3682.7200,3715.6699,3644.7600,3655.0400,3655.0400,4.886440e+09,4.886440e+09,-27.6800,-0.752,3671.82330,"September 26, 22",-0.00752,^GSPC,NaN,NaN,NaN,NaN,NaN,NaN,2.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
master.to_csv("app/data_storage/"+file_name, index=False)